Copyright &copy; 2023-2024 Praneeth Vadlapati

## Setup:

Example of .env: 
```bash
# Groq API to use LLMs - https://console.groq.com/keys
# Groq is preferred for fast responses
LM_PROVIDER_BASE_URL=https://api.groq.com/openai/v1
LM_API_KEY=
LM_MODEL=

```

Installing packages:
```bash
pip install openai python-dotenv pandas
```

## Loading an LLM

In [1]:
import json
import os
import sys
import io

import pandas as pd
from common_functions import get_lm_response, extract_data, \
    print_progress, print_error, model, display_md, load_env

print(f'Model: {model}')

Model: llama-3.1-70b-versatile


## A. 	Creation of a table of tools
## B. 	Incorporating new tools into the table

In [ ]:
tools_data = [
	# {
	# 	'toolID': 'weathr1',
	# 	'tool_name': 'Weather API',
	# 	'tool_desc': 'Provides weather information',
	# 	'python_function': '''
	# 		def main(location: str) -> str:
	# 			return '30 degrees Fahrenheit, cloudy'
	# 	'''.strip(),
	# 	'input_desc': { 'location': 'should be a string in the format of "city, country"' },
	# 	'pip_packages': ['requests'],
	# 	'keywords': None,
	# },
	{
		'toolID': 'calc',
		'tool_name': 'Calculator',
		'tool_desc': 'Performs basic arithmetic operations as a calculator',
		'python_function': '''
def main(a: float, o: str, b: float) -> float:
	if o == '+':
		return a + b
	elif o == '-':
		return a - b
	elif o == '*':
		return a * b
	elif o == '/':
		return a / b
	else:
		return 'Invalid operator'

		'''.strip(),
		'input_desc': { 'a': 'first number as a float', 'o': 'operator like +,-,*,/', 'b': 'second number as a float' },
		'pip_packages': None,
		'keywords': None,
	},
	# exec - code interpreter of python
	{
		'toolID': 'exec',
		'tool_name': 'Code Interpreter',
		'tool_desc': 'Runs any Python code and returns what is printed',
		'python_function': '''
import io
import sys

def main(code: str) -> str:
	old_stdout = sys.stdout
	new_stdout = io.StringIO()
	sys.stdout = new_stdout
	try:
		exec(code)
		output = new_stdout.getvalue()
		if output and output.strip():
			return output
		else:
			raise Exception('No output')
	except Exception as e:
		return str(e)
	finally:
		sys.stdout = old_stdout

		'''.strip(),
		'input_desc': { 'code': 'source code in Python with "print" statements to print the outputs' },
		'pip_packages': None,
		'keywords': None,
	},
]


sample_val_prompt_template = '''
Act as a QA engineer and provide a sample value to test the code.

Input description:
{input_desc}
---
Return the sample value in json format like:
<sample_value>
{{ "<key>": "<value>" }}
</sample_value>
'''.strip()
# Code:
# {python_function}
# ---

keywords_prompt_template = '''
Tool details:
Tool Name: {tool_name}
Tool Description: {tool_desc}
---
Provide keywords that can be used to search for this tool.
Keywords should be separated by commas.
Sample response:
`Weather, Temperature, Cloudy`
'''.strip()

# create a DataFrame using the given columns without data
tool_df = pd.DataFrame(columns=tools_data[0].keys())
all_tool_keywords = set()

for tool_data in tools_data:
	# install the required packages, and test the code by generating a sample value
	pip_packages = tool_data.get('pip_packages', []) or []
	for package in pip_packages:
		os.system(f'pip install {package} > /dev/null 2>&1')

	python_function = tool_data['python_function']
	if 'def main(' not in python_function:
		raise Exception('The code should contain a function named "main"')

	try:
		sample_val_response = get_lm_response(sample_val_prompt_template.format(
												# python_function=tool_data['python_function']))
												input_desc=tool_data['input_desc']))
		sample_values = extract_data(sample_val_response, tag='sample_value')
		sample_values = json.loads(sample_values)
		# print(f'Sample value: {sample_values}')

		# execute the function using sample values
		local_vars = {}
		exec(python_function, local_vars)
		main_func = local_vars.get('main')
		if not main_func:
			raise Exception('Function "main" not found in the provided code')
		sample_response = main_func(**sample_values)
		if not sample_response:
			raise Exception('Function did not return any value')
		# print(f'Sample response: {sample_response}')

		# generate the keywords for the tool
		keywords_response = get_lm_response(keywords_prompt_template.format(**tool_data))
		keywords = extract_data(keywords_response)
		keywords = [keyword.lower().strip() for keyword in keywords.split(',')]
		tool_data['keywords'] = keywords
		all_tool_keywords.update(keywords)

		# add the tool to the DataFrame
		new_row = pd.DataFrame([tool_data])
		tool_df = pd.concat([tool_df, new_row], ignore_index=True)
		print(f'{tool_data["tool_name"]} added successfully')
	except Exception as e:
		print(f'Error adding {tool_data["tool_name"]}')
		print(e)

tool_df

Calculator added successfully
Code Interpreter added successfully


,toolID,tool_name,tool_desc,python_function,input_desc,pip_packages,keywords
0,calc,Calculator,Performs basic arithmetic operations as a calc...,"def main(a: float, o: str, b: float) -> float:...","{'a': 'first number as a float', 'o': 'operato...",None,"[calculator, math operations, arithmetic, addi..."
1,exec,Code Interpreter,Runs any Python code and returns what is printed,import io\nimport sys\n\ndef main(code: str) -...,"{'code': 'source code in Python with ""print"" s...",None,"[python interpreter, code execution, execute p..."


## C. 	User query processing using tools

In [ ]:
query_required_prompt_template = '''
User Query: {user_query}
---
Does the user query require any of the tools?
Respond with `YES` or `NO`, in uppercase inside the backticks.
Note: Consider to use tools for anything in which calculations, executions, etc. can be involved in any way.
	Use tools for even the most basic tasks like simple calculation, counting, code execution, etc.
'''.strip()

tool_keywords_prompt_template = '''
User Query: {user_query}
Available tool keywords:
{available_tool_keywords}
---
Provide the keywords that the user query can match with.
Select all the relevant keywords from the list.
Keywords should be separated by commas.
Sample response:
`Weather, Temperature`
'''.strip()

tool_selection_prompt_template = '''
Relevant Tools:
{relevant_tools}
---
User Query:
{user_query}
---
Select the tool that can be used to answer the user query.
Provide the tool ID of the selected tool exactly as shown in the list.
Return ID inside backticks.
Sample response:
`weathr1`
'''.strip()

input_args_prompt_template = '''
Tool Name: {tool_name}
Tool Description: {tool_desc}
---
Input Description:
{input_desc}
---
User Query:
{user_query}
---
Provide the input arguments for the selected tool.
Input arguments should be in json format.
Sample response:
`{{ "location": "New York, US" }}`
'''.strip()

user_response_prompt_template = '''
Tool name:
{tool_name}
Input details:
{input_desc}
Tool input arguments:
{input_args}
Tool Response:
{tool_response}
---
User Query: {user_query}
---
Provide the response to the user query.
Return the response inside backticks.
Sample response:
`The weather in New York is 30 degrees Fahrenheit and cloudy`
'''.strip()

execution_results_df = pd.DataFrame(columns=['Query', 'Tools_used', 'Response'])

user_queries = [
    # 'Hi',
    'What is 1+1?',
	'What is 1-1?',
	'How many “r”s are in “strawberry?”',
	# 'Show the news',
]

for user_query in user_queries:
	query_required_response = get_lm_response(
		query_required_prompt_template.format(user_query=user_query))
	query_required = extract_data(query_required_response)
	query_required = 'YES' in query_required.upper()

	if not query_required:
		print('No tools required for the user query')
		continue

	tool_keywords_response = get_lm_response(tool_keywords_prompt_template.format(
								available_tool_keywords=', '.join(all_tool_keywords)))
	tool_keywords = extract_data(tool_keywords_response)
	tool_keywords = [keyword.lower().strip() for keyword in tool_keywords.split(',')]
	# print(f'Tool Keywords: {tool_keywords}')

	# fetch tools based that contain the selected keywords
	relevant_tools_df = tool_df[tool_df['keywords'].apply(  # select tools
			lambda x: any(keyword in x for keyword in tool_keywords))]
	if relevant_tools_df.empty:
		# raise Exception('No tools found for the user query')
		print('No tools available for the user query')
		selected_tool_ID = None
  
	else:
		# print(f'Relevant Tools: {relevant_tools_df}')

		# selecting a tool from the relevant tools
		relevant_tools_str = '\n'.join(relevant_tools_df.apply(  # [['toolID', 'tool_name', 'tool_desc']]
								lambda x: f'{x["toolID"]}: {x["tool_name"]} - {x["tool_desc"]}', axis=1))
		tool_selection_response = get_lm_response(tool_selection_prompt_template.format(
									relevant_tools=relevant_tools_str, user_query=user_query))
		selected_tool_ID = extract_data(tool_selection_response)
		selected_tool_ID = selected_tool_ID.strip()
		# print(f'Selected Tool ID: {selected_tool_ID}')

		# generating input arguments based on selected tool and user query
		selected_tool = tool_df[tool_df['toolID'] == selected_tool_ID].iloc[0]
		input_args_response = get_lm_response(input_args_prompt_template.format(
								tool_name=selected_tool['tool_name'],
								tool_desc=selected_tool['tool_desc'],
								user_query=user_query,
								input_desc=str(selected_tool['input_desc']),
							))
		input_args = extract_data(input_args_response)
		input_args = json.loads(input_args)
		# print(f'Input Args: {input_args}')

		# execute the function using input values
		local_vars = {}
		exec(selected_tool['python_function'], local_vars)
		main_func = local_vars.get('main')
		tool_response = main_func(**input_args)
		# print(f'Tool Response: {tool_response}')

		# provide the response to the user query
		user_response_response = get_lm_response(user_response_prompt_template.format(
									tool_name=selected_tool['tool_name'],
									input_args=input_args,
									tool_response=tool_response,
        							input_desc=str(selected_tool['input_desc']),
									user_query=user_query))
		user_response = extract_data(user_response_response)
		print(f'User Response: `{user_response}`')

	# add execution results
	new_execution_row = pd.DataFrame([{
		'Query': user_query,
		'Tools_used': selected_tool_ID,
		'Response': user_response
	}])
	execution_results_df = pd.concat([execution_results_df, new_execution_row], ignore_index=True)

execution_results_df

User Response: `1 + 1 = 2.0`
User Response: `The result of 1-1 is 0`
User Response: `The number of "r"s in "strawberry" is: 3`


,Query,Tools_used,Response
0,What is 1+1?,calc,8\n0\n
1,What is 1-1?,calc,8\n0\n
2,How many “r”s are in “strawberry?”,exec,8\n0\n
